Bài 3. Viết và triển khai Smart Contract đơn giản bằng Python
Mục tiêu: Làm quen với Smart Contract và công cụ Web3.py trong phạm vi đơn giản nhất.
Yêu cầu:
 Dùng mã Solidity đã trình bày trong chương 4 để viết một hợp đồng lưu thông điệp.
 Biên dịch hợp đồng bằng solcx.
 Dùng Web3.py kết nối Ganache và triển khai hợp đồng.
Gửi thông điệp mới và đọc lại thông điệp đã lưu.

In [1]:
from solcx import compile_source, install_solc
from web3 import Web3

# Cài phiên bản Solidity compiler
install_solc('0.8.0')

# 1. Kết nối Ganache
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
assert w3.is_connected(), "Không kết nối được Ganache!"

# 2. Lấy tài khoản đầu tiên làm người deploy
w3.eth.default_account = w3.eth.accounts[0]

# 3. Mã nguồn Solidity
contract_source_code = '''
pragma solidity ^0.8.0;

contract messagestorage {
    string public message;

    constructor(string memory initialMessage) {
        message = initialMessage;
    }

    function setMessage(string memory newMessage) public {
        message = newMessage;
    }
}
'''

# 4. Compile source code
compiled_sol = compile_source(contract_source_code, solc_version='0.8.0')
contract_id, contract_interface = compiled_sol.popitem()

# 5. Lấy ABI và bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

# 6. Tạo contract object
HelloMessage = w3.eth.contract(abi=abi, bytecode=bytecode)

# 7. Deploy contract (khởi tạo message ban đầu)
tx_hash = HelloMessage.constructor("Xin chào blockchain!").transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
contract_address = tx_receipt.contractAddress
print(f"Địa chỉ hợp đồng: {contract_address}")

# 8. Tạo instance contract đã triển khai
contract_instance = w3.eth.contract(address=contract_address, abi=abi)

# 9. Đọc thông điệp ban đầu
print("Thông điệp ban đầu:", contract_instance.functions.message().call())

# 10. Gửi thông điệp mới
tx_hash2 = contract_instance.functions.setMessage("Thông điệp mới từ Python!").transact()
w3.eth.wait_for_transaction_receipt(tx_hash2)

# 11. Đọc lại thông điệp sau khi thay đổi
print("Thông điệp mới:", contract_instance.functions.message().call())


Địa chỉ hợp đồng: 0x0681f29Bec129df73c949b9bB5109D961d2C3582
Thông điệp ban đầu: Xin chào blockchain!
Thông điệp mới: Thông điệp mới từ Python!
